In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
%matplotlib inline

### Get HGNC Mapper

In [2]:
hgnc = pd.read_csv('../data/hgnc_alias_list.txt', sep='\t')
#hgnc.head()

hgnc = hgnc[hgnc['Approved symbol'].apply(lambda x: 'withdrawn' not in x)]

# get the original keys
hgnc_original_keys = hgnc['Approved symbol'].unique()

# drop a lot of columns for efficiency
hgnc = hgnc[list(hgnc.columns)[0:6]].drop('Status', axis=1)

# filter out Nan synonyms (not helpful)
hgnc_syn_list = hgnc[~ hgnc.Synonyms.isna()]
hgnc_prev_symb_list = hgnc[~ hgnc['Previous symbols'].isna()]

print(hgnc_syn_list.shape)
print(hgnc_syn_list.columns)
display(hgnc_syn_list.head())

print(hgnc_prev_symb_list.shape)
print(hgnc_prev_symb_list.columns)
display(hgnc_prev_symb_list.head())

#convert the synonyms column to a list
# convert these lists to pd.Series
# merge with original dataframe
#drop old synonyms column 
# melt the new columns into rows


current_syn_list = hgnc_syn_list.Synonyms.apply(lambda x: x.split(',')) \
    .apply(pd.Series) \
    .merge(hgnc, left_index = True, right_index = True) \
    .drop(["Synonyms"], axis = 1) \
    .melt(id_vars = ['HGNC ID', 'Approved symbol', 'Approved name', 'Previous symbols'], value_name = "synonym") 

current_syn_list = current_syn_list[~ current_syn_list.synonym.isna()]
current_syn_list.synonym = current_syn_list.synonym.apply(lambda x: x.replace(' ',''))


current_syn_list.head()

prev_symb_list = hgnc_prev_symb_list['Previous symbols'].apply(lambda x: x.split(',')) \
    .apply(pd.Series) \
    .merge(hgnc, left_index = True, right_index = True) \
    .drop(['Previous symbols'], axis = 1) \
    .melt(id_vars = ['HGNC ID', 'Approved symbol', 'Approved name', 'Synonyms'], value_name = "synonym") 

prev_symb_list = prev_symb_list[~ prev_symb_list.synonym.isna()]
prev_symb_list.synonym = prev_symb_list.synonym.apply(lambda x: x.replace(' ',''))


prev_symb_list.head()

hgnc_mapper = dict(zip(current_syn_list['synonym'], current_syn_list['Approved symbol']))
# add in HGNC ID mapper
hgnc_mapper.update(dict(zip(current_syn_list['HGNC ID'], current_syn_list['Approved symbol'])))
hgnc_mapper_previous = dict(zip(prev_symb_list['synonym'], prev_symb_list['Approved symbol']))

trouble_list = list(filter(lambda x: hgnc_mapper[x] != hgnc_mapper_previous[x], set(hgnc_mapper.keys())&set(hgnc_mapper_previous.keys())))

hand_coded = {'RAGE':'MOK', 'SGK2':'SGK2', 'SGK196':'SGK196', 'MAPK3':'MAPK3'}

hgnc_mapper_previous.update(hgnc_mapper) #overwrite the previous symbol conflicts

hgnc_mapper = hgnc_mapper_previous
hgnc_mapper.update({x:x for x in hgnc_original_keys}) #keep the identify maps
hgnc_mapper.update(hand_coded) # overwrite the trouble list

(21435, 5)
Index(['HGNC ID', 'Approved symbol', 'Approved name', 'Previous symbols',
       'Synonyms'],
      dtype='object')


,HGNC ID,Approved symbol,Approved name,Previous symbols,Synonyms
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,"NCRNA00181, A1BGAS, A1BG-AS",FLJ23569
2,HGNC:24086,A1CF,APOBEC1 complementation factor,NaN,"ACF, ASP, ACF64, ACF65, APOBEC1CF"
4,HGNC:7,A2M,alpha-2-macroglobulin,NaN,"FWP007, S863-7, CPAMD5"
6,HGNC:23336,A2ML1,alpha-2-macroglobulin like 1,CPAMD9,"FLJ25179, p170"
12,HGNC:30005,A3GALT2,"alpha 1,3-galactosyltransferase 2",A3GALT2P,"IGBS3S, IGB3S"


(11702, 5)
Index(['HGNC ID', 'Approved symbol', 'Approved name', 'Previous symbols',
       'Synonyms'],
      dtype='object')


,HGNC ID,Approved symbol,Approved name,Previous symbols,Synonyms
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,"NCRNA00181, A1BGAS, A1BG-AS",FLJ23569
6,HGNC:23336,A2ML1,alpha-2-macroglobulin like 1,CPAMD9,"FLJ25179, p170"
9,HGNC:8,A2MP1,alpha-2-macroglobulin pseudogene 1,A2MP,NaN
12,HGNC:30005,A3GALT2,"alpha 1,3-galactosyltransferase 2",A3GALT2P,"IGBS3S, IGB3S"
13,HGNC:18149,A4GALT,"alpha 1,4-galactosyltransferase (P blood group)",P1,"A14GALT, Gb3S, P(k)"


### Get Uniprot mapper

In [3]:
uni_mapper = pd.read_csv('../data/network/HUMAN_9606_idmapping.dat', sep='\t', header=None)
uni_mapper = uni_mapper[uni_mapper[1] == 'Gene_Name'].set_index(0)[2].to_dict()

next(iter(uni_mapper.items()))

('P31946', 'YWHAB')

In [4]:
uni_rev = {y:x for x,y in uni_mapper.items()}

In [5]:
print(len(uni_mapper))
print(len(uni_rev))

146363
26468


### Extra mappers from UniProt, e.g. ENSG

In [6]:
t = pd.read_csv('../data/network/HUMAN_9606_idmapping_selected.tab', sep='\t', header=None, low_memory=False)
display(t.shape)
display(t.head())

entrez_to_uniprot_number = t.set_index(2)[0].to_dict()
ensg_to_uniprot_number = t.set_index(18)[0].to_dict()
#ensg_mapper = t[]
#t = uni_mapper[uni_mapper[1]                                   == 'Gene_Name'].set_index(0)[2].to_dict()

#next(iter(uni_mapper.items()))



(169389, 22)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,P31946,1433B_HUMAN,7529,NP_003395.1; NP_647539.1; XP_016883528.1,4507949; 377656702; 67464628; 1345590; 1034625...,2BQ0:A; 2BQ0:B; 2C23:A; 4DNK:A; 4DNK:B; 5N10:A...,GO:0005737; GO:0005829; GO:0070062; GO:0005925...,UniRef100_P31946,UniRef90_P31946,UniRef50_P31946,...,9606,601289,Hs.643544,8515476; 14702039; 11780052; 15489334; 2357255...,X57346; AK292717; AL008725; CH471077; CH471077...,CAA40621.1; BAF85406.1; -; EAW75893.1; EAW7589...,ENSG00000166913,ENST00000353703; ENST00000372839,ENSP00000300161; ENSP00000361930,11996670; 12364343; 12437930; 12468542; 124825...
1,P62258,1433E_HUMAN,7531,NP_006752.1,374074368; 62131678; 67464424; 194379794; 2210...,2BR9:A; 3UAL:A; 3UBW:A; 6EIH:A,GO:0090724; GO:0005737; GO:0005829; GO:0070062...,UniRef100_P62258,UniRef90_P62258,UniRef50_P62258,...,9606,605066,Hs.513851,7644510; 8858348; 8684458; 20417184; 14702039;...,U20972; U54778; U43399; U43430; U28936; AB0171...,AAC50175.1; AAC50710.1; AAC50625.1; AAD00026.1...,ENSG00000108953; ENSG00000274474,ENST00000264335; ENST00000571732; ENST00000616...,ENSP00000264335; ENSP00000461762; ENSP00000481...,15838597; 11782387; 12657644; 14966136; 153649...
2,Q04917,1433F_HUMAN,7533,NP_003396.1,4507951; 437363; 83754686; 83754699; 83754700;...,2C63:A; 2C63:B; 2C63:C; 2C63:D; 2C74:A; 2C74:B,GO:0005737; GO:0005829; GO:0070062; GO:0098978...,UniRef100_Q04917,UniRef90_Q04917,UniRef50_P62258,...,9606,113508,Hs.226755,8218406; 1578511; 8561965; 8812417; 15461802; ...,L20422; X80536; X78138; X57345; D78577; S80794...,AAA35483.1; CAA56676.1; CAA55017.1; CAA40620.1...,ENSG00000128245,ENST00000248975,ENSP00000248975,10206237; 11121172; 11996670; 12176995; 124801...
3,P61981,1433G_HUMAN,7532,NP_036611.2,6016838; 380764684; 635576381; 82407956; 82407...,2B05:A; 2B05:B; 2B05:C; 2B05:D; 2B05:E; 2B05:F...,GO:0005829; GO:0070062; GO:0005925; GO:0016020...,UniRef100_P61981,UniRef90_P61981,UniRef50_P61981,...,9606,605356; 617665,Hs.744840,10433554; 10486217; 12853948; 15489334; 235725...,AF142498; AB024334; CR541904; CR541925; AC0063...,AAD48408.1; BAA85184.1; CAG46702.1; CAG46723.1...,ENSG00000170027,ENST00000307630,ENSP00000306330,11824616; 11996670; 12364343; 12482592; 150572...
4,P31947,1433S_HUMAN,2810,NP_006133.1,436408756; 306991738; 969812714; 350610438; 35...,1YWT:A; 1YWT:B; 1YZ5:A; 1YZ5:B; 3IQJ:A; 3IQU:A...,GO:0005829; GO:0070062; GO:0005615; GO:0005739...,UniRef100_P31947,UniRef90_P31947,UniRef50_P61981,...,9606,601290,Hs.523718,1390337; 8515476; 9659898; 16710414; 15489334;...,M93010; X57348; AF029081; AF029082; CR541905; ...,AAA59546.1; CAA40623.1; AAC52029.1; AAC52030.1...,ENSG00000175793,ENST00000339276,ENSP00000340989,10969776; 12582028; 12730237; 12787309; 145172...


### Kinase Master List (to filter)

In [7]:
kmast = pd.read_excel('../data/KINASESmasterlist_w_Aliases.xlsx')
kmast.head()

,Uniprot Protein,MS Gene,RNAseq Gene,RNAseq Accession,Family,Mouse Uniprot Protein,Mouse RNAseq gene,Mouse RNAseq Accession,Kinome Render Tree Name,Aliases (Conservative),...,Aliases,description,other_designations,Entrez_Symbol,Old_Name,Entrez_Synonyms,Entrez_other_designations,Unnamed: 18,Gene Symbol,Gene Symbol and Synonyms
0,AAK1,AAK1,AAK1,22848.0,Other,Aak1,Aak1,269774,AAK1,"AAK1,",...,NaN,AP2 associated kinase 1,adaptor-associated kinase 1,AAK1,AAK1,KIAA1048|MGC138170,adaptor-associated kinase 1,NaN,AAK1,AAK1
1,AAPK1,PRKAA1,PRKAA1,5562.0,CAMK,Aapk1,Prkaa1,105787,AMPK[alpha]1,"PRKAA1, AMPKa1",...,"AMPK, AMPKa1","protein kinase, AMP-activated, alpha 1 catalyt...","5'-AMP-activated protein kinase, catalytic alp...",PRKAA1,AMPKa1,AMPK|AMPKa1|MGC33776|MGC57364,"5'-AMP-activated protein kinase, catalytic alp...",NaN,PRKAA1,"AMPK, PRKAA1, AMPKa1"
2,AAPK2,PRKAA2,PRKAA2,5563.0,CAMK,Aapk2,Prkaa2,108079,AMPK[alpha]2,"PRKAA2, AMPK2, AMPKa2, PRKAA",...,"AMPK, AMPK2, AMPKa2, PRKAA","protein kinase, AMP-activated, alpha 2 catalyt...","5'-AMP-activated protein kinase, catalytic alp...",PRKAA2,AMPKa2,AMPK|AMPK2|PRKAA,"5'-AMP-activated protein kinase, catalytic alp...",NaN,PRKAA2,"AMPK2, AMPK, PRKAA, AMPKa2, PRKAA2"
3,ABL1,ABL1,ABL1,25.0,TK,Abl1,Abl1,11350,Abl,"ABL1,ABL, JTK7, bcr/abl, c-ABL, c-ABL1, v-abl",...,"ABL, JTK7, bcr/abl, c-ABL, c-ABL1, p150, v-abl","ABL proto-oncogene 1, non-receptor tyrosine ki...",Abelson tyrosine-protein kinase 1|bcr/c-abl on...,ABL1,ABL,ABL|JTK7|c-ABL|p150|v-abl,Abelson murine leukemia viral (v-abl) oncogene...,NaN,ABL1,"c-ABL1, p150, ABL, JTK7, v-abl, bcr/abl, c-ABL..."
4,ABL2,ABL2,ABL2,27.0,TK,Abl2,Abl2,11352,Arg,"ABL2,ABLL, ARG",...,"ABLL, ARG","ABL proto-oncogene 2, non-receptor tyrosine ki...","abelson-related gene protein|c-abl oncogene 2,...",ABL2,ARG,ABLL|ARG,Abelson murine leukemia viral (v-abl) oncogene...,NaN,ABL2,"ABL2, ABLL, ARG"


In [8]:
[x for x,y in hgnc_mapper.items() if 'SGK' in y]

['SGK', 'SGK2', 'SGKL', 'C8orf44-SGK3', 'SGK1', 'SGK3', 'SGK196']

In [9]:
hgnc_mapper['MAPK3']

'MAPK3'

In [10]:
dict(zip(kmast['MS Gene'], kmast['Uniprot Protein']))

{'AAK1': 'AAK1',
 'PRKAA1': 'AAPK1',
 'PRKAA2': 'AAPK2',
 'ABL1': 'ABL1',
 'ABL2': 'ABL2',
 'TNK2': 'ACK1',
 'ACVR1B': 'ACV1B',
 'ACVR1C': 'ACV1C',
 'ACVRL1': 'ACVL1',
 'ACVR1': 'ACVR1',
 'ADCK1': 'ADCK1',
 'ADCK2': 'ADCK2',
 'ADCK3': 'ADCK3',
 'ADCK4': 'ADCK4',
 'ADCK5': 'ADCK5',
 'ADK': 'ADK',
 'AGK': 'AGK',
 'AKT1': 'AKT1',
 'AKT2': 'AKT2',
 'AKT3': 'AKT3',
 'ALK': 'ALK',
 'ALPK1': 'ALPK1',
 'ALPK2': 'ALPK2',
 'ALPK3': 'ALPK3',
 'AMHR2': 'AMHR2',
 'ANKK1': 'ANKK1',
 'NPR1': 'ANPRA',
 'NPR2': 'ANPRB',
 'ARAF': 'ARAF',
 'ADRBK1': 'ARBK1',
 'ADRBK2': 'ARBK2',
 'ATM': 'ATM',
 'ATR': 'ATR',
 'AURKB': 'AURKB',
 'AURKC': 'AURKC',
 'ACVR2A': 'AVR2A',
 'ACVR2B': 'AVR2B',
 'BCKDK': 'BCKD',
 'BLK': 'BLK',
 'BMP2K': 'BMP2K',
 'BMPR2': 'BMPR2',
 'BMPR1A': 'BMR1A',
 'BMPR1B': 'BMR1B',
 'BMX': 'BMX',
 'BRAF': 'BRAF',
 'BRD2': 'BRD2',
 'BRD3': 'BRD3',
 'BRD4': 'BRD4',
 'BRSK1': 'BRSK1',
 'BRSK2': 'BRSK2',
 'BTK': 'BTK',
 'BUB1': 'BUB1',
 'BUB1B': 'BUB1B',
 'CAMKV': 'CAMKV',
 'CDK11A': 'CD11A',
 'CD

In [11]:
hard_coded = {'ACV1B':'ACVR1B', 'PDPK2':'PDK2', 'SGK110':'SBK3', 'SGK223':'PRAG1'}
hgnc_mapper.update(hard_coded)
kinome_set = set(hgnc_mapper[x] for x in kmast['MS Gene'] if 'GYK' != x and 'SGK494' != x)# if x in hgnc_mapper.keys())


uniprot_to_hgnc_map_helper = {y:hgnc_mapper[x] for x,y in zip(kmast['MS Gene'], kmast['Uniprot Protein']) if 'GYK' != x and 'SGK494' != x}
uniprot_to_hgnc_map_helper.update({'RAC1':'RAC1'})
#hgnc_mapper.update(uniprot_to_hgnc_map_helper)
# ^ this causes a lot of collision issues e.g. mapk3 and mapkapk3

len(kinome_set)

567

### Filter our uniprot queries down

In [12]:
print(len(set(kmast['Uniprot Protein'])))
print(len(set(kmast['Symbol'])))
print(len(set(kmast['Symbol'])|set(kmast['Uniprot Protein'])))

570
499
744


In [13]:
uniprot_targets = set(kmast['Uniprot Protein'])
uniprot_targets

{'AAK1',
 'AAPK1',
 'AAPK2',
 'ABL1',
 'ABL2',
 'ACK1',
 'ACV1B',
 'ACV1C',
 'ACVL1',
 'ACVR1',
 'ADCK1',
 'ADCK2',
 'ADCK3',
 'ADCK4',
 'ADCK5',
 'ADK',
 'AGK',
 'AKT1',
 'AKT2',
 'AKT3',
 'ALK',
 'ALPK1',
 'ALPK2',
 'ALPK3',
 'AMHR2',
 'ANKK1',
 'ANPRA',
 'ANPRB',
 'ARAF',
 'ARBK1',
 'ARBK2',
 'ATM',
 'ATR',
 'AURKB',
 'AURKC',
 'AVR2A',
 'AVR2B',
 'BCKD',
 'BLK',
 'BMP2K',
 'BMPR2',
 'BMR1A',
 'BMR1B',
 'BMX',
 'BRAF',
 'BRD2',
 'BRD3',
 'BRD4',
 'BRSK1',
 'BRSK2',
 'BTK',
 'BUB1',
 'BUB1B',
 'CAMKV',
 'CD11A',
 'CD11B',
 'CDC7',
 'CDK1',
 'CDK10',
 'CDK12',
 'CDK13',
 'CDK14',
 'CDK15',
 'CDK16',
 'CDK17',
 'CDK18',
 'CDK19',
 'CDK2',
 'CDK20',
 'CDK3',
 'CDK4',
 'CDK5',
 'CDK6',
 'CDK7',
 'CDK8',
 'CDK9',
 'CDKL1',
 'CDKL2',
 'CDKL3',
 'CDKL4',
 'CDKL5',
 'CHK1',
 'CHK2',
 'CHKA',
 'CHKB',
 'CLK1',
 'CLK2',
 'CLK3',
 'CLK4',
 'CSF1R',
 'CSK',
 'CSK21',
 'CSK22',
 'CSK23',
 'CSKP',
 'CTRO',
 'DAPK1',
 'DAPK2',
 'DAPK3',
 'DCK',
 'DCLK1',
 'DCLK2',
 'DCLK3',
 'DDR1',
 'DDR2',
 'DGKQ

In [14]:
print(len(uni_mapper))

146363


### Start importing sources

In [15]:
data_sources = {}
data_preproc = {}

# hardcode MAPK3 due to MAPKAPK3 issues
id_dict = lambda x: hgnc_mapper[x]  if x in hgnc_mapper.keys()\
                                        else 'MAPK3' if x == 'P27361'\
                                        else uniprot_to_hgnc_map_helper[x] if x in uniprot_to_hgnc_map_helper.keys()\
                                        else uniprot_to_hgnc_map_helper[uni_mapper[x]] if x in uni_mapper.keys() and uni_mapper[x] in uniprot_to_hgnc_map_helper.keys()\
                                        else hgnc_mapper[uni_mapper[x]] if x in uni_mapper.keys() and uni_mapper[x] in hgnc_mapper.keys()\
                                        else None

#else uniprot_to_hgnc_map_helper[uni_mapper[entrez_to_uniprot_number[x]]] if x in entrez_to_uniprot_number.keys() and entrez_to_uniprot_number[x] in uni_mapper.keys() and uni_mapper[entrez_to_uniprot_number[x]] in uniprot_to_hgnc_map_helper.keys()\
#else uniprot_to_hgnc_map_helper[uni_mapper[ensg_to_uniprot_number[x]]] if x in ensg_to_uniprot_number.keys() and ensg_to_uniprot_number[x] in uni_mapper.keys() and uni_mapper[ensg_to_uniprot_number[x]] in uniprot_to_hgnc_map_helper.keys()\
                                        

In [16]:
hippie = pd.read_csv('../data/network/hippie_current.txt', sep='\t', header=None)

data_sources['hippie'] = hippie
data_preproc['hippie'] = {
    'columns':[0,2],
    'maps':[
        lambda x: id_dict(x.split('_')[0]),
        lambda x: id_dict(x.split('_')[0])
    ]
}

print(hippie.columns)
hippie.head()

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')


,0,1,2,3,4,5
0,AL1A1_HUMAN,216,AL1A1_HUMAN,216,0.76,"experiments:in vivo,Two-hybrid;pmids:12081471,..."
1,ITA7_HUMAN,3679,ACHA_HUMAN,1134,0.73,"experiments:in vivo,Affinity Capture-Western,a..."
2,NEB1_HUMAN,55607,ACTG_HUMAN,71,0.65,"experiments:in vitro,in vivo;pmids:9362513,120..."
3,SRGN_HUMAN,5552,CD44_HUMAN,960,0.63,"experiments:in vivo;pmids:9334256,16189514,167..."
4,GRB7_HUMAN,2886,ERBB2_HUMAN,2064,0.90,"experiments:in vitro,in vivo,Reconstituted Com..."


In [17]:
phosphosite_substrate = pd.read_csv('../data/network/Kinase_Substrate_Dataset.txt', sep='\t', skiprows=2)

data_sources['phosphosite_substrate'] = phosphosite_substrate
data_preproc['phosphosite_substrate'] = {
    'columns':['GENE','SUB_GENE'],
    'maps':[
        lambda x: id_dict(x.upper()),
        lambda x: id_dict(x.upper())
    ]
}

print(phosphosite_substrate.columns)
display(phosphosite_substrate.head())


phosphosite_regulatory = pd.read_csv('../data/network/Regulatory_sites.txt', sep='\t', skiprows=3, error_bad_lines=False)

#data_sources['phosphosite_regulatory'] =  phosphosite_regulatory
data_preproc['phosphosite_regulatory'] = {
    'columns':['GENE','SUBSTRATE'],
    'maps':[
        lambda x: id_dict(x.upper()),
        lambda x: id_dict(x.upper())
    ]
}

#print(phosphosite_regulatory.columns)
#display(phosphosite_regulatory.head())

Index(['GENE', 'KINASE', 'KIN_ACC_ID', 'KIN_ORGANISM', 'SUBSTRATE',
       'SUB_GENE_ID', 'SUB_ACC_ID', 'SUB_GENE', 'SUB_ORGANISM', 'SUB_MOD_RSD',
       'SITE_GRP_ID', 'SITE_+/-7_AA', 'DOMAIN', 'IN_VIVO_RXN', 'IN_VITRO_RXN',
       'CST_CAT#'],
      dtype='object')


,GENE,KINASE,KIN_ACC_ID,KIN_ORGANISM,SUBSTRATE,SUB_GENE_ID,SUB_ACC_ID,SUB_GENE,SUB_ORGANISM,SUB_MOD_RSD,SITE_GRP_ID,SITE_+/-7_AA,DOMAIN,IN_VIVO_RXN,IN_VITRO_RXN,CST_CAT#
0,Pak2,PAK2,Q64303,rat,MEK1,170851.0,Q01986,Map2k1,rat,S298,448284,RtPGRPLsSYGMDSR,Pkinase,,X,9128; 98195
1,Pak2,PAK2,Q64303,rat,PRKD1,85421.0,Q9WTQ1,Prkd1,rat,S203,449896,GVRRRRLsNVsLTGL,NaN,X,,NaN
2,Pak2,PAK2,Q64303,rat,prolactin,5617.0,P01236,PRL,human,S207,451732,LHCLRRDsHKIDNYL,Hormone_1,,X,NaN
3,Pak2,PAK2,Q64303,rat,prolactin,24683.0,P01237,Prl,rat,S206,451732,IRCLRRDsHKVDNYL,Hormone_1,,X,NaN
4,EIF2AK1,HRI,Q9BQI3,human,eIF2-alpha,54318.0,P68101,Eif2s1,rat,S52,447635,MILLSELsRRRIRSI,S1,,X,3597; 9721; 3398; 5199


In [18]:
hprd = pd.read_csv('../data/network/HPRD_ALL_BINARY_PROTEIN_PROTEIN_INTERACTIONS.txt', sep='\t', header=None)

data_sources['hprd'] = hprd
data_preproc['hprd'] = {
    'columns':[0,3],
    'maps':[
        lambda x: id_dict(x.upper()),
        lambda x: id_dict(x.upper())
    ]
}

print(hprd.columns)
hprd.head()

Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')


,0,1,2,3,4,5,6,7
0,ALDH1A1,1,NP_000680.2,ALDH1A1,1,NP_000680.2,in vivo;yeast 2-hybrid,"1,208,147,116,189,510"
1,ITGA7,2761,NP_001138468.1,CHRNA1,7,NP_001034612.1,in vivo,10910772
2,PPP1R9A,16000,NP_060120.2,ACTG1,17,NP_001605.1,in vitro;in vivo,"936,251,312,052,877"
3,SRGN,1513,NP_002718.2,CD44,115,NP_000601.3,in vivo,9334256
4,GRB7,3311,NP_005301.2,ERBB2,1281,NP_004439.2,in vitro;in vivo,9079677


In [19]:
entrez_to_uniprot_number['31']

'Q7Z5W8'

In [20]:
reactome = pd.read_csv('../data/network/reactome.homo_sapiens.interactions.tab-delimited.txt', sep='\t')

data_sources['reactome'] = reactome
data_preproc['reactome'] = {
    'columns': ['# Interactor 1 uniprot id','Interactor 2 uniprot id'],
    'maps': [
        lambda x: id_dict(x.split(':')[1]),
        lambda x: id_dict(x.split(':')[1])
    ]
}


print(reactome.columns)
reactome.head()

Index(['# Interactor 1 uniprot id', 'Interactor 1 Ensembl gene id',
       'Interactor 1 Entrez Gene id', 'Interactor 2 uniprot id',
       'Interactor 2 Ensembl gene id', 'Interactor 2 Entrez Gene id',
       'Interaction type', 'Interaction context', 'Pubmed references'],
      dtype='object')


,# Interactor 1 uniprot id,Interactor 1 Ensembl gene id,Interactor 1 Entrez Gene id,Interactor 2 uniprot id,Interactor 2 Ensembl gene id,Interactor 2 Entrez Gene id,Interaction type,Interaction context,Pubmed references
0,uniprotkb:Q9Y287,ENSEMBL:ENSG00000136156,entrezgene/locuslink:9445,uniprotkb:Q9Y287,ENSEMBL:ENSG00000136156,entrezgene/locuslink:9445,physical association,reactome:R-HSA-976871,14690516|10391242
1,uniprotkb:P37840,ENSEMBL:ENSG00000145335,entrezgene/locuslink:6622,uniprotkb:P37840,ENSEMBL:ENSG00000145335,entrezgene/locuslink:6622,physical association,reactome:R-HSA-1247852,24243840
2,uniprotkb:P0DJI8,ENSEMBL:ENSG00000173432,entrezgene/locuslink:6288,uniprotkb:P0DJI8,ENSEMBL:ENSG00000173432,entrezgene/locuslink:6288,physical association,reactome:R-HSA-976898,19393650|103558
3,uniprotkb:P06727,ENSEMBL:ENSG00000110244,entrezgene/locuslink:337,uniprotkb:P06727,ENSEMBL:ENSG00000110244,entrezgene/locuslink:337,physical association,reactome:R-HSA-976889,15146166
4,uniprotkb:P01160,ENSEMBL:ENSG00000175206,entrezgene/locuslink:4878,uniprotkb:P01160,ENSEMBL:ENSG00000175206,entrezgene/locuslink:4878,physical association,reactome:R-HSA-976987,2142465|2945573


In [21]:
biogrid = pd.read_csv('../data/network/BIOGRID-ORGANISM-Homo_sapiens-3.5.168.tab2.txt', low_memory=False, sep='\t')

data_sources['biogrid'] = biogrid
data_preproc['biogrid'] = {
    'columns': ['Official Symbol Interactor A','Official Symbol Interactor B'],
    'maps': [
        lambda x: id_dict(x),
        lambda x: id_dict(x)
    ]
}

print(biogrid.columns)
biogrid.head()

Index(['#BioGRID Interaction ID', 'Entrez Gene Interactor A',
       'Entrez Gene Interactor B', 'BioGRID ID Interactor A',
       'BioGRID ID Interactor B', 'Systematic Name Interactor A',
       'Systematic Name Interactor B', 'Official Symbol Interactor A',
       'Official Symbol Interactor B', 'Synonyms Interactor A',
       'Synonyms Interactor B', 'Experimental System',
       'Experimental System Type', 'Author', 'Pubmed ID',
       'Organism Interactor A', 'Organism Interactor B', 'Throughput', 'Score',
       'Modification', 'Phenotypes', 'Qualifications', 'Tags',
       'Source Database'],
      dtype='object')


,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,Pubmed ID,Organism Interactor A,Organism Interactor B,Throughput,Score,Modification,Phenotypes,Qualifications,Tags,Source Database
0,103,6416,2318,112315,108607,-,-,MAP2K4,FLNC,JNKK|JNKK1|MAPKK4|MEK4|MKK4|PRKMK4|SAPKK-1|SAP...,...,9006895,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
1,117,84665,88,124185,106603,-,-,MYPN,ACTN2,CMD1DD|CMH22|MYOP|RCM4,...,11309420,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
2,183,90,2339,106605,108625,-,-,ACVR1,FNTA,ACTRI|ACVR1A|ACVRLK2|ALK2|FOP|SKR1|TSRI,...,8599089,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
3,278,2624,5371,108894,111384,-,-,GATA2,PML,DCML|IMD21|MONOMAC|NFE1B,...,10938104,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID
4,418,6118,6774,112038,112651,RP4-547C9.3,-,RPA2,STAT3,REPA2|RP-A p32|RP-A p34|RPA32,...,10875894,9606,9606,Low Throughput,-,-,-,-,-,BIOGRID


In [22]:
i2d = pd.read_csv('../data/network/i2d.2_9.txt', sep='\t')

data_sources['i2d'] = i2d
data_preproc['i2d'] = {
    'columns': ['SwissProt1','SwissProt2'],
    'maps': [
        lambda x: id_dict(x),
        lambda x: id_dict(x)
    ]
}

print(i2d.columns)
i2d.head()

Index(['Dataset', 'SwissProt1', 'SwissProt2'], dtype='object')


,Dataset,SwissProt1,SwissProt2
0,SOURAV_MAPK_LOW,P63000,A0AUZ9
1,IntAct,Q96CV9,A0AUZ9
2,BioGrid,P0CG48,A0AV96
3,IntAct_Mouse,P62258,A0AV96
4,IntAct_Mouse,P63104,A0AV96


In [23]:
mint = pd.read_csv('../data/network/species_human.txt', sep='\t', header=None)

# drop empty interaction columns
mint = mint[mint[0].apply(lambda x: ':' in x)]
mint = mint[mint[1].apply(lambda x: ':' in x)]

data_sources['mint'] = mint
data_preproc['mint'] = {
    'columns': [0,1],
    'maps':[
        lambda x: id_dict(x.split(':')[1]),
        lambda x: id_dict(x.split(':')[1])
    ]
}

print(mint.columns)
mint.head()

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,uniprotkb:Q72547,uniprotkb:Q72547,intact:EBI-7484755|intact:MINT-8208544,intact:EBI-7484755|intact:MINT-8208544,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,"psi-mi:""MI:0030""(cross-linking study)",Nishitsuji et al. (2011),pubmed:22004763|imex:IM-16791|mint:MINT-820698...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0471""(MINT)",intact:EBI-7484777|mint:MINT-8206988|imex:IM-1...,intact-miscore:0.40
1,uniprotkb:Q72547,uniprotkb:Q72547,intact:EBI-7484755|intact:MINT-8208544,intact:EBI-7484755|intact:MINT-8208544,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,"psi-mi:""MI:0030""(cross-linking study)",Nishitsuji et al. (2011),pubmed:22004763|imex:IM-16791|mint:MINT-820698...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0471""(MINT)",intact:EBI-7484916|mint:MINT-8207003|imex:IM-1...,intact-miscore:0.40
2,uniprotkb:Q72547,uniprotkb:Q72547,intact:EBI-7484755|intact:MINT-8208544,intact:EBI-7484755|intact:MINT-8208544,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,psi-mi:q72547_9hiv1(display_long)|uniprotkb:po...,"psi-mi:""MI:0030""(cross-linking study)",Nishitsuji et al. (2011),pubmed:22004763|imex:IM-16791|mint:MINT-820698...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0471""(MINT)",intact:EBI-7484965|mint:MINT-8207016|imex:IM-1...,intact-miscore:0.40
3,uniprotkb:Q90VU7,uniprotkb:Q900A7,intact:EBI-7460704|intact:MINT-5281653,intact:EBI-7460739|intact:MINT-8208185,psi-mi:q90vu7_9hiv1(display_long)|uniprotkb:ne...,psi-mi:q900a7_9hiv1(display_long)|uniprotkb:ta...,"psi-mi:""MI:0416""(fluorescence microscopy)",Sugiyama et al. (2011),pubmed:21970979|imex:IM-16787|mint:MINT-820620...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,taxid:11676(9hiv1)|taxid:11676(Human immunodef...,"psi-mi:""MI:0403""(colocalization)","psi-mi:""MI:0471""(MINT)",intact:EBI-7460732|mint:MINT-8206271|imex:IM-1...,intact-miscore:0.27
4,uniprotkb:Q2Q067,uniprotkb:Q13015,intact:EBI-9675545,intact:EBI-6269719,psi-mi:q2q067_9dela(display_long)|uniprotkb:HB...,psi-mi:af1q_human(display_long)|uniprotkb:MLLT...,"psi-mi:""MI:0397""(two hybrid array)",Simonis et al. (2012),imex:IM-22977|pubmed:22458338,taxid:11908(humt-)|taxid:11908(Human T-lymphot...,taxid:9606(human)|taxid:9606(Homo sapiens),"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0471""(MINT)",intact:EBI-9675551|imex:IM-22977-4,intact-miscore:0.49


In [24]:
string_database_score_filter = 900

string = pd.read_csv('../data/network/9606.protein.links.full.v11.0.txt', sep=' ')
string = string[string['database']>=string_database_score_filter]
string_alias = pd.read_csv('../data/network/human.uniprot_2_string.2018.tsv', sep='\t', header=None)

string_to_uniprot = dict(zip(string_alias[2], string_alias[1].apply(lambda x: x.split('|')[1].split('_')[0])))

data_sources['string'] = string
data_preproc['string'] = {
    'columns': ['protein1','protein2'],
    'maps': [
        lambda x: id_dict(string_to_uniprot[x]) if x in string_to_uniprot.keys() else None,
        lambda x: id_dict(string_to_uniprot[x]) if x in string_to_uniprot.keys() else None
    ]
}

print(string.columns)
display(string.head())
print(string_alias.columns)
display(string_alias.head())

Index(['protein1', 'protein2', 'neighborhood', 'neighborhood_transferred',
       'fusion', 'cooccurence', 'homology', 'coexpression',
       'coexpression_transferred', 'experiments', 'experiments_transferred',
       'database', 'database_transferred', 'textmining',
       'textmining_transferred', 'combined_score'],
      dtype='object')


,protein1,protein2,neighborhood,neighborhood_transferred,fusion,cooccurence,homology,coexpression,coexpression_transferred,experiments,experiments_transferred,database,database_transferred,textmining,textmining_transferred,combined_score
15,9606.ENSP00000000233,9606.ENSP00000432568,0,0,0,0,0,0,62,0,63,900,0,0,94,909
16,9606.ENSP00000000233,9606.ENSP00000427900,0,0,0,0,0,0,0,0,131,900,0,0,56,910
40,9606.ENSP00000000233,9606.ENSP00000354878,0,0,0,0,0,0,0,0,131,900,0,0,56,910
52,9606.ENSP00000000233,9606.ENSP00000405926,0,0,0,0,0,0,63,0,0,900,0,64,58,906
78,9606.ENSP00000000233,9606.ENSP00000314615,0,0,0,0,0,61,63,0,262,900,0,473,317,971


Int64Index([0, 1, 2, 3, 4], dtype='int64')


,0,1,2,3,4
0,9606,Q03135|CAV1_HUMAN,9606.ENSP00000339191,100.0,370.0
1,9606,P35125|UBP6_HUMAN,9606.ENSP00000460380,100.0,2941.0
2,9606,P41161|ETV5_HUMAN,9606.ENSP00000306894,100.0,1045.0
3,9606,Q6KF10|GDF6_HUMAN,9606.ENSP00000287020,100.0,928.0
4,9606,Q495C1|RN212_HUMAN,9606.ENSP00000389709,100.0,610.0


## Network Assembly

In [26]:
network = pd.DataFrame(columns=['Node1', 'Node2'])
network.head()

,Node1,Node2


#### Before string: 4599 edges (400 self-self)
#### No filter on string: 39520 edges
#### Database >= 900 filter on string: 5062
#### Database >= 800 filter on string: 5294
#### Database >= 600 filter on string: 5478

In [27]:
network = pd.DataFrame(columns=['Node1', 'Node2'])

for source_name, data in data_sources.items():
    
    #if source_name in ['hippie', 'phosphosite_substrate', 'hprd', 'biogrid']:
    #    continue
    
    print('Adding', source_name, 'to network')
    
    left_col, right_col = data_preproc[source_name]['columns']
    left_map, right_map = data_preproc[source_name]['maps']
    
    data = data[[left_col, right_col]].dropna(axis=0)
    
    #print(data.dtypes)#[left_col])
    
    to_add = pd.DataFrame(columns=network.columns)
    to_add['Node1'] = data[left_col].apply(left_map)
    to_add['Node2'] = data[right_col].apply(right_map)
    to_add.dropna(axis=0, inplace = True)
   
    network = pd.concat([network, to_add], ignore_index=True)
    
    #break
    
# drop obvious duplicates
network.drop_duplicates(inplace = True)

# drop items not in the target kinome set
network = network[(network['Node1'].isin(kinome_set)) & (network['Node2'].isin(kinome_set))]

network_cop = copy(network[['Node2','Node1']])
network_cop.columns = ['Node1', 'Node2']

# drop a -> b b -> a redundancies 
network = network.append(network_cop, ignore_index=True).drop_duplicates()
network = network[network['Node1']<network['Node2']]

#for i in network.index:
#    network.loc[i] = sorted(network.loc[i])
    
#network.drop_duplicates(inplace=True)

network

Adding hippie to network
Adding phosphosite_substrate to network
Adding hprd to network
Adding reactome to network
Adding biogrid to network
Adding i2d to network
Adding mint to network
Adding string to network


,Node1,Node2
2,MST1R,YES1
3,TYRO3,YES1
11,FGR,SRC
16,ATR,FLT1
19,LYN,PRKCD
22,LYN,SYK
29,LYN,MAP4K1
30,PDGFRB,PIK3CA
34,PHKG1,PHKG2
36,PDGFRA,PDGFRB


## Full Comparison

In [ ]:
understudied = pd.read_csv('../data/dark_kinases.csv', sep=',')
understudied.head()

In [ ]:
understudied_set = set(understudied.symbol)

In [ ]:
kyla_network = pd.read_csv('../data/KIN_edges_no_weights.txt', header=None, sep='\t')

rna_to_MS = kmast.set_index('RNAseq Gene')['MS Gene'].to_dict()
hgnc_mapper.update({'KPCA':'PRKCA','KSYK':'SYK', 'VGFR2':'KDR', 'KPCT':'PRKCQ'})
kyla_network[0] = kyla_network[0].apply(lambda x: hgnc_mapper[x] if x in hgnc_mapper.keys() else hgnc_mapper[rna_to_MS[x]])
kyla_network[1] = kyla_network[1].apply(lambda x: hgnc_mapper[x] if x in hgnc_mapper.keys() else hgnc_mapper[rna_to_MS[x]])

kyla_network

In [ ]:
new_edges = set(zip(network['Node1'], network['Node2']))
kyla_edges = set(zip(kyla_network[0], kyla_network[1]))

In [ ]:
print('kyla: ', len(kyla_edges))
print('new: ', len(new_edges))
print('overlap: ',len(new_edges & kyla_edges))

In [ ]:
new_tot = set(network['Node1']) | set(network['Node2'])
kyla_tot = set(kyla_network[0]) | set(kyla_network[1])

In [ ]:
print('kyla: ', len(kyla_tot))
print('new: ', len(new_tot))
print(len(new_hippie_tot & kyla_tot))

In [ ]:
len(set(kyla_network[0].unique()) | set(kyla_network[1].unique()))

In [ ]:
np.sum(kyla_network[0] == kyla_network[1])

In [ ]:
for i in kyla_network.index:
    kyla_network.loc[i] = sorted(kyla_network.loc[i])

In [ ]:
kyla_tuples = list(zip(kyla_network[0], kyla_network[1]))
new_tuples = list(zip(network['Node1'], network['Node2']))

### This used to be > 2700

### Best: 2805

In [ ]:
## missing is now 272

In [ ]:
missing = set(kyla_tuples) - set(new_tuples)
l1, l2 = zip(*missing)
problem_kin = list(l1)+list(l2)

In [ ]:
set(problem_kin) & understudied_set

In [ ]:
problem_kin

In [29]:
network.to_csv('../data/kin_unweighted.csv', sep='\t', header=None, index=False)

In [ ]:
from collections import Counter

c = Counter(problem_kin)
c.most_common()

In [ ]:
from collections import Counter

c = Counter(problem_kin)
c.most_common()

### Weighting Functions